In [1]:
import pyscipopt
import itertools

In [8]:
n = 17
cities = list(range(n))

distance_matrix = [
 [9999, 3, 5, 48, 48, 8, 8, 5, 5, 3, 3, 0, 3, 5, 8, 8, 5],
 [3, 9999, 3, 48, 48, 8, 8, 5, 5, 0, 0, 3, 0, 3, 8, 8, 5],
 [5, 3, 9999, 72, 72, 48, 48, 24, 24, 3, 3, 5, 3, 0, 48, 48, 24],
 [48, 48, 74, 9999, 0, 6, 6, 12, 12, 48, 48, 48, 48, 74, 6, 6, 12],
 [48, 48, 74, 0, 9999, 6, 6, 12, 12, 48, 48, 48, 48, 74, 6, 6, 12],
 [8, 8, 50, 6, 6, 9999, 0, 8, 8, 8, 8, 8, 8, 50, 0, 0, 8],
 [8, 8, 50, 6, 6, 0, 9999, 8, 8, 8, 8, 8, 8, 50, 0, 0, 8],
 [5, 5, 26, 12, 12, 8, 8, 9999, 0, 5, 5, 5, 5, 26, 8, 8, 0],
 [5, 5, 26, 12, 12, 8, 8, 0, 9999, 5, 5, 5, 5, 26, 8, 8, 0],
 [3, 0, 3, 48, 48, 8, 8, 5, 5, 9999, 0, 3, 0, 3, 8, 8, 5],
 [3, 0, 3, 48, 48, 8, 8, 5, 5, 0, 9999, 3, 0, 3, 8, 8, 5],
 [0, 3, 5, 48, 48, 8, 8, 5, 5, 3, 3, 9999, 3, 5, 8, 8, 5],
 [3, 0, 3, 48, 48, 8, 8, 5, 5, 0, 0, 3, 9999, 3, 8, 8, 5],
 [5, 3, 0, 72, 72, 48, 48, 24, 24, 3, 3, 5, 3, 9999, 48, 48, 24],
 [8, 8, 50, 6, 6, 0, 0, 8, 8, 8, 8, 8, 8, 50, 9999, 0, 8],
 [8, 8, 50, 6, 6, 0, 0, 8, 8, 8, 8, 8, 8, 50, 0, 9999, 8],
 [5, 5, 26, 12, 12, 8, 8, 0, 0, 5, 5, 5, 5, 26, 8, 8, 9999]
]

In [9]:
model = pyscipopt.Model("TSP")

# Create variables
x = {}
for i in range(n):
    for j in range(n):
        if i == j:
            x[i,j] = model.addVar(lb=0, ub=0, vtype="B", name="x(%s,%s)"%(i,j))
        else:
            x[i,j] = model.addVar(lb=0, ub=1, vtype="B", name="x(%s,%s)"%(i,j))

# Create objective function
model.setObjective(pyscipopt.quicksum(distance_matrix[i][j]*x[i,j] for i in range(n) for j in range(n)), sense="minimize")

In [10]:
def Combination(n:int):
    Q = []
    
    for i in range(2, n):
        Q.append(list(itertools.combinations(cities, i)))
    
    result = []
    for i in Q:
        for j in i:
            result.append(list(j))
    
    return result

In [11]:
# Create constraints
# Assignment constraints 1
for i in range(n):
    model.addCons(pyscipopt.quicksum(x[i,j] for j in range(n)) == 1)

# Assignment constraints 2
for j in range(n):
    model.addCons(pyscipopt.quicksum(x[i,j] for i in range(n)) == 1)

# Subtour-Elimination constraints (DFJ formulation)
Q = Combination(n)

print(len(Q))
print(Q)

for each_q in Q:
    model.addCons(pyscipopt.quicksum(x[i,j]for i in each_q for j in each_q if i != j) <= len(each_q)-1)


56
[[0, 1], [0, 2], [0, 3], [0, 4], [0, 5], [1, 2], [1, 3], [1, 4], [1, 5], [2, 3], [2, 4], [2, 5], [3, 4], [3, 5], [4, 5], [0, 1, 2], [0, 1, 3], [0, 1, 4], [0, 1, 5], [0, 2, 3], [0, 2, 4], [0, 2, 5], [0, 3, 4], [0, 3, 5], [0, 4, 5], [1, 2, 3], [1, 2, 4], [1, 2, 5], [1, 3, 4], [1, 3, 5], [1, 4, 5], [2, 3, 4], [2, 3, 5], [2, 4, 5], [3, 4, 5], [0, 1, 2, 3], [0, 1, 2, 4], [0, 1, 2, 5], [0, 1, 3, 4], [0, 1, 3, 5], [0, 1, 4, 5], [0, 2, 3, 4], [0, 2, 3, 5], [0, 2, 4, 5], [0, 3, 4, 5], [1, 2, 3, 4], [1, 2, 3, 5], [1, 2, 4, 5], [1, 3, 4, 5], [2, 3, 4, 5], [0, 1, 2, 3, 4], [0, 1, 2, 3, 5], [0, 1, 2, 4, 5], [0, 1, 3, 4, 5], [0, 2, 3, 4, 5], [1, 2, 3, 4, 5]]


In [13]:
model.writeProblem("TSP-DFJ.lp")
model.optimize()
status = model.getStatus()
print(f"Solution status: {status}")
if status == "optimal":
    print(f"Objective value = {model.getObjVal()}")
    for i in range(n):
        for j in range(n):
            if model.getVal(x[i,j]) > 0:
                print(f"{x[i,j]} = {model.getVal(x[i,j])}")

wrote problem to file /home/ytsao/code/Optimization-models/Code/Python/TSP-DFJ.lp
Solution status: optimal
Objective value = 117.0
x(0,3) = 1.0
x(1,2) = 1.0
x(2,0) = 1.0
x(3,4) = 1.0
x(4,5) = 1.0
x(5,1) = 1.0
